In [1]:
import pandas as pd
from os import getcwd, listdir, makedirs
from os.path import join, isfile, exists, abspath, pardir
from sys import path
# from pandas_profiling import ProfileReport

##### Configs

In [2]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, "data")
scripts_dir = join(parent_dir, "src", "scripts")
data_file = join(data_dir, "publications.csv")

In [3]:
path.append(scripts_dir)
from connect import Neo4jConnection

##### Load data

In [4]:
df = pd.read_csv(data_file)
df.head(3)

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,Document Type,Publication Stage,Access Type,Source,EID
0,"Gautam A., Crandall J.W., Goodrich M.A.",57218202833;7004904337;7005513246;,Self-assessment of Proficiency of Intelligent ...,2021,Advances in Intelligent Systems and Computing,1210 AISC,NaN,NaN,108,113,...,"Computer Science Department, Brigham Young Uni...","Gautam, A., Computer Science Department, Brigh...","Autonomous systems, although capable of perfor...",Environment; Goal(s); Intelligent agents; Prof...,Drones; Human engineering; Intelligent systems...,Conference Paper,Final,NaN,Scopus,2-s2.0-85088238482
1,"de Moura Oliveira P.B., Hedengren J.D., Boaven...",6508306234;9277159100;6507358470;,Bridging theory to practice: Feedforward and c...,2021,Lecture Notes in Electrical Engineering,695 LNEE,NaN,NaN,23,32,...,"INESC-TEC Technology and Science, Campus da FE...","de Moura Oliveira, P.B., INESC-TEC Technology ...",Practice is of the essence in Engineering cour...,NaN,Automation; Cascade control systems; Computati...,Conference Paper,Final,NaN,Scopus,2-s2.0-85091306533
2,"Hajimirzaie S.M., Hotchkiss R.H.",53879700900;26642910500;,Development of sediment management guidelines ...,2020,Journal of Hydraulic Engineering,146,12,02520004,NaN,NaN,...,"Task Committee Secretary and Lead Engineer, Op...","Hajimirzaie, S.M., Task Committee Secretary an...",Forum papers are thought-provoking opinion pie...,NaN,Hydraulics; Editorial board; Sediment manageme...,Review,Final,NaN,Scopus,2-s2.0-85091917202


In [7]:
# profile = ProfileReport(df, title="Publications")
# profile.to_widgets()

In [5]:
# database = "sdm"
# driver = Neo4jConnection(uri="bolt://localhost:7687", user=None, pwd=None, database=database)

# result = driver.query("""
# Match (n) return n
# """)

# r = result.single()

# list(result)

In [6]:
df.columns.to_list()

['Authors',
 'Author(s) ID',
 'Title',
 'Year',
 'Source title',
 'Volume',
 'Issue',
 'Art. No.',
 'Page start',
 'Page end',
 'Page count',
 'Cited by',
 'DOI',
 'Link',
 'Affiliations',
 'Authors with affiliations',
 'Abstract',
 'Author Keywords',
 'Index Keywords',
 'Document Type',
 'Publication Stage',
 'Access Type',
 'Source',
 'EID']

In [7]:
journals_cols = ['Year', 'Source title', 'Volume']
keyword_cols = ['Author Keywords', 'Index Keywords']
affiliation_cols = ['Affiliations']
authors_cols = ['Authors', 'Author(s) ID', 'Affiliations', 'Authors with affiliations', 'Author Keywords']
document_cols = ['Author(s) ID', 'Title', 'Source title', 'Art. No.', 'Cited by', 'DOI', 'Abstract', 'Index Keywords', 'Document Type']

In [51]:
def cast_columns(df, cols, __type):
    df[cols] = df[cols].astype(__type)
    return df

#### Journals

In [8]:
journals_df = df[journals_cols]

In [11]:
journals_df.dtypes

Year             int64
Source title    object
Volume          object
dtype: object

In [12]:
journals_df.head()

,Year,Source title,Volume
0,2021,Advances in Intelligent Systems and Computing,1210 AISC
1,2021,Lecture Notes in Electrical Engineering,695 LNEE
2,2020,Journal of Hydraulic Engineering,146
3,2020,Journal of Computing and Information Science i...,20
4,2020,Telematics and Informatics,55


In [34]:
# journals_df[['Source title', 'Volume']] = journals_df[['Source title', 'Volume']].astype(pd.StringDtype())
journals_df = cast_columns(journals_df, ['Source title', 'Volume'], pd.StringDtype())
journals_df.rename(columns={'Year': 'year', 'Source title': 'name', 'Volume': 'volume'}, inplace=True)
journals_df.dtypes

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_31871/4243683562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  journals_df[['Source title', 'Volume']] = journals_df[['Source title', 'Volume']].astype(pd.StringDtype())
/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_31871/4243683562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  journals_df.rename(columns={'Year': 'year', 'Source title': 'name', 'Volume': 'volume'}, inplace=True)


year       int64
name      string
volume    string
dtype: object

In [43]:
journals_df[journals_df['volume'].isnull()].head()

,year,name,volume
22,2020,Proceedings - 2020 IEEE 21st International Con...,<NA>
50,2020,Proceedings - 2020 IEEE 34th International Par...,<NA>
95,2020,Journal of Composite Materials,<NA>
96,2020,Computing in Science and Engineering,<NA>
101,2020,International Journal of Robotics Research,<NA>


In [49]:
journals_df.to_csv(join(data_dir, "journals.csv"), index=False)

##### Keywords

In [50]:
keywords_df = df[keyword_cols]
keywords_df.dtypes

Author Keywords    object
Index Keywords     object
dtype: object

In [52]:
keywords_df = cast_columns(keywords_df, keyword_cols, pd.StringDtype())
keywords_df.dtypes

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_31871/4139907851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols] = df[cols].astype(__type)


Author Keywords    string
Index Keywords     string
dtype: object

In [53]:
keywords_df.head()

,Author Keywords,Index Keywords
0,Environment; Goal(s); Intelligent agents; Prof...,Drones; Human engineering; Intelligent systems...
1,<NA>,Automation; Cascade control systems; Computati...
2,<NA>,Hydraulics; Editorial board; Sediment manageme...
3,Computer-aided engineering; Data-driven engine...,Compressors; Data handling; Structural design;...
4,Dating; Emerging adults; Mobile phones; Uses a...,Entertainment; A-RINGS; External pressures; Ro...


In [54]:
keywords_df

Author Keywords    True
Index Keywords     True
dtype: bool